In [2]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_runner
import importlib
import flg_model
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import os
import itertools
import glob

fast_mode = (fls.env=='local')
clear_data = False

MainProcess


In [3]:
# # Prepare datasets
# if fls.env=='vast':
#     if not fast_mode:
#         fls.download_kaggle_dataset('jeroencottaar/byu-many-models/', fls.result_dir + '/many_full/', skip_download=clear_data)
#         fls.download_kaggle_dataset('jeroencottaar/byu-many-models-abbreviated/', fls.result_dir + '/many_abbr/', skip_download=clear_data)
#     else:
#         if clear_data:
#             fls.remove_and_make_dir(fls.result_dir + '/many_full/')
#             fls.remove_and_make_dir(fls.result_dir + '/many_abbr/')

Dataset URL: https://www.kaggle.com/datasets/jeroencottaar/byu-many-models/versions/
License(s): unknown


100%|██████████| 9.00G/9.00G [00:56<00:00, 171MB/s] 



Downloaded metadata to /flagellar/results//many_full/dataset-metadata.json
Dataset URL: https://www.kaggle.com/datasets/jeroencottaar/byu-many-models-abbreviated/versions/
License(s): unknown


100%|██████████| 253M/253M [00:02<00:00, 115MB/s]  



Downloaded metadata to /flagellar/results//many_abbr/dataset-metadata.json


In [ ]:
#fls.profiling=True
import git 
repo = git.Repo(search_parent_directories=True)
git_commit_id = repo.head.object.hexsha
for i in itertools.count(start=0):
    print('Starting seed ', i)
    r = flg_runner.baseline_runner(fast_mode=fast_mode)
    #r.use_missing_value = True
    # r.label = 'Test ensemble'
    # r.modifier_dict = dict()
    # r.modifier_dict['n_ensemble'] = flg_runner.pm(1, lambda r:(r.integers(4,5)).item(), flg_runner.yolo)    
    # r.base_model = flg_model.ThreeStepModelLabelBased()
    # r.base_model.train_data_selector.datasets = ['tom']
    # r.base_model.run_in_parallel = False
    # if fast_mode: r.base_model.step1Labels.n_epochs = 1
    r.git_commit_id = git_commit_id
    r.env = fls.env
    r.seed = i
    #r.base_model.train_data_selector.datasets = 'tom'
    #r.base_model.run_in_parallel = False
    if r.use_missing_value:
        base_filename = r.label + '_' + str(r.seed) + 'M_' + git_commit_id[:8]   
    else:
        base_filename = r.label + '_' + str(r.seed) + '_' + git_commit_id[:8]   
        if len(glob.glob(fls.result_dir + '/many_full/' + r.label + '_' + str(r.seed) + '_900' + '*'))>=1:
             print('Skipping')
             continue
    r.run()

    score_str = str(r.cv_score[2])[2:5]
    print(str(r.cv_score)[1:5])
    output_file_full = fls.result_dir + '/many_full/' + base_filename + '_' + score_str +' _f.pickle'
    output_file_abbr = fls.result_dir + '/many_abbr/' + base_filename + '_' + score_str +' _a.pickle'
    
    fls.dill_save(output_file_full, r)
    r.trained_model = 0
    fls.dill_save(output_file_abbr, r)
    if not fast_mode and fls.env=='vast':
       fls.upload_kaggle_dataset(fls.result_dir + '/many_full/')
       fls.upload_kaggle_dataset(fls.result_dir + '/many_abbr/')

Starting seed  0
Skipping
Starting seed  1
Skipping
Starting seed  2
Skipping
Starting seed  3
Skipping
Starting seed  4
Skipping
Starting seed  5
Skipping
Starting seed  6
Skipping
Starting seed  7
Skipping
Starting seed  8
Skipping
Starting seed  9
Skipping
Starting seed  10
Skipping
Starting seed  11
Skipping
Starting seed  12
Skipping
Starting seed  13
Skipping
Starting seed  14
Skipping
Starting seed  15
Skipping
Starting seed  16
Skipping
Starting seed  17
Skipping
Starting seed  18
Skipping
Starting seed  19
Skipping
Starting seed  20
Skipping
Starting seed  21
Skipping
Starting seed  22
Skipping
Starting seed  23
Skipping
Starting seed  24
1701 230
1701 230
{'seed': 24, 'n_ensemble': 2, 'scale_approach': 1, 'scale_percentile_value': 3.8103546312782877, 'moving_ratio': 0.2149475638654384, 'scale_std_clip_value': 2.506399767270148, 'blur_z': 3, 'img_size': 640, 'n_epochs': 49, 'lr0': 0.0012377862786311516, 'use_best_epoch': False, 'model_name': 'yolov8m', 'use_pretrained_weights'

SpawnPoolWorker-1
CUDA_VISIBLE_DEVICES= 0
CompletedProcess(args=['pip', 'install', 'albumentations'], returncode=0)
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/opt/conda/lib/python3.11/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


Will process approximately 4455 slices for training
Processing training motors:   0%|          | 0/405 [00:00<?, ?it/s]
Clearing pytorch
-1.9386058 1.8795774
-1.9877445 1.9271705
-1.9869574 1.9279575
-1.9895937 1.929068
-1.9880939 1.9305671
-1.9880233 1.9343911
-1.9868553 1.93932
-1.9861944 1.9437488
-1.9845154 1.949203
-1.9811182 1.9526004
-1.9304426 1.906523
-1.6463925 1.9199058
-1.6641488 1.9437819
-1.6601332 1.9509785
-1.6585275 1.9589646
-1.6538666 1.9636252
-1.6535742 1.9703201
-1.6523919 1.9747123
-1.6519057 1.9816353
-1.652343 1.9876578
-1.6534215 1.9898175
-1.6346687 1.9669158
-1.563889 2.2542949
-1.5904803 2.29472
-1.589503 2.2920146
-1.5872152 2.2906268
-1.5918454 2.2859964
-1.5958004 2.2820415
-1.600209 2.2776327
-1.5968212 2.2736902
-1.5917759 2.2714324
-1.5878686 2.2680645
-1.5573468 2.2264385
-1.8172343 1.8894887
-1.8495731 1.9246213
-1.8444595 1.9262596
-1.8444628 1.9297316
-1.8409888 1.9332054
-1.8416723 1.9325218
-1.8385197 1.9321992
-1.8393079 1.9314109
-1.8420177 1.